In [9]:
g = 9.80665

m1 = 1.786614971490255
m2 = 0.08939794594871456

l1 = 0.007967562168493848
l2 = 0.06

L1 = 0.075
L2 = 0.12

# J1 = pendulum_params["J1"]
# J2 = pendulum_params["J2"]

b1 = 0.0
b2 = 0.0

# J2_hat = J2 + m2 * l2 * l2
# J0_hat = J1 + m1 * l1 * l1 + m2 * L1 * L1

J0_hat = 0.00283041
J2_hat = 0.000322832

In [10]:
import numpy as np

# Linearized furuta pendulum from
# https://www.hindawi.com/journals/jcse/2011/528341/

denominator = J0_hat * J2_hat - (m2**2.0) * (L1**2.0) * (l2**2.0)

A32 = (g * (m2**2.0) * (l2**2.0) * L1) / denominator
A33 = (-b1 * J2_hat) / denominator
A34 = (-b2 * m2 * l2 * L1) / denominator

A42 = (g * m2 * l2 * J0_hat) / denominator
A43 = (-b1 * m2 * l2 * L1) / denominator
A44 = (-b2 * J0_hat) / denominator

B31 = (J2_hat) / denominator
B41 = (m2 * L1 * l2) / denominator
B32 = (m2 * L1 * l2) / denominator
B42 = (J0_hat) / denominator

A = np.array([[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, A32, A33, A34], [0.0, A42, A43, A44]])
B = np.array([[0.0], [0.0], [B31], [B41]])
# B32 and B42 not used, as I assumed no disturbance (tau2 = 0)

print(A, B)

[[  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]
 [  0.          28.14324672  -0.          -0.        ]
 [  0.         198.00834495  -0.          -0.        ]] [[  0.        ]
 [  0.        ]
 [429.34978933]
 [535.02580806]]


In [11]:
theta1_weight = 0.0
theta2_weight = 10.0
dtheta1_weight = 1.0
dtheta2_weight = 1.0
u_weight = 1.0

Q = np.array(
    [
        [theta1_weight, 0.0, 0.0, 0.0],
        [0.0, theta2_weight, 0.0, 0.0],
        [0.0, 0.0, dtheta1_weight, 0.0],
        [0.0, 0.0, 0.0, dtheta2_weight],
    ]
)
R = np.array([u_weight])

In [12]:
from control import lqr, dlqr, ss

# K, S, E = lqr(A, B, Q, R)
C = np.identity(4)
# D = np.zeros(4).transpose()
D = np.array([[0.0], [0.0], [0.0], [0.0]])

# dt = 0.006666667
dt = 1/330
dsys = ss(A, B, C, D, dt)
K, S, E = dlqr(dsys, Q, R)

sys = ss(A, B, C, D)
Kc, S, E = lqr(sys, Q, R)

print(f"{Kc[0,0]}, {Kc[0,1]}, {Kc[0,2]}, {Kc[0,3]}")
print(f"{K[0,0]}, {K[0,1]}, {K[0,2]}, {K[0,3]}")

print(E)

-4.1801204715813414e-14, 24.126627150719628, -0.9999999999999858, 2.1193630064897135
0.0, 0.3700797393830494, 0.0, -1.5222021172687217e-15
[-6.86189589e+02+0.j          1.75881174e-14+0.j
 -9.18726355e+00+4.18865651j -9.18726355e+00-4.18865651j]


In [13]:
from numpy.linalg import matrix_rank

n = A.shape[0]
controllability_matrix = np.hstack([np.linalg.matrix_power(A, i) @ B for i in range(n)])
rank = matrix_rank(controllability_matrix)
print(f"Rank: {rank}, Number of States: {n}")

Rank: 4, Number of States: 4
